In [1]:
from transformers import BertTokenizer, AutoTokenizer, AutoModel

In [2]:
# from_pretrained(类方法): Instantiate a [`~tokenization_utils_base.PreTrainedTokenizerBase`] (or a derived class) from a predefined tokenizer.
# tokenizer = BertTokenizer.from_pretrained(
#     pretrained_model_name_or_path='bert-base-chinese',
# )
# This is a generic tokenizer class that will be instantiated as one of the tokenizer classes of the library when created with the [`AutoTokenizer.from_pretrained`] class method.
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path='bert-base-chinese')  # 与上等价
tokenizer

BertTokenizerFast(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [3]:
text = '月光的[UNK][PAD][CLS]新希望[EOS]<eop>'

# 未添加新tokens前的编码效果
print(tokenizer.convert_tokens_to_ids(['月光', '希望', '[EOS]', '<eop>']))
print(tokenizer.tokenize(text))
print(len(tokenizer.get_vocab()))

[100, 100, 100, 100]
['月', '光', '的', '[UNK]', '[PAD]', '[CLS]', '新', '希', '望', '[', '[UNK]', ']', '<', 'e', '##op', '>']
21128


In [4]:
# Add a list of new tokens to the tokenizer class.
tokenizer.add_tokens(new_tokens=['月光', '希望'])

# Add a dictionary of special tokens (eos, pad, cls, etc.) to the encoder and link them to class attributes.
# If special tokens are NOT in the vocabulary, they are added to it (indexed starting from the last index of the current vocabulary).
print(tokenizer.eos_token, tokenizer.eos_token_id)
tokenizer.add_special_tokens(
    # Keys should be in the list of predefined special attributes: [`bos_token`, `eos_token`, `unk_token`, `sep_token`, `pad_token`, `cls_token`, `mask_token`, `additional_special_tokens`].
    special_tokens_dict={'eos_token': '[EOS]', 'additional_special_tokens': ["<eop>", "<eod>"]})
print(tokenizer.eos_token, tokenizer.eos_token_id)
print(len(tokenizer.get_vocab()))
print(tokenizer.special_tokens_map)
print(tokenizer.additional_special_tokens, tokenizer.additional_special_tokens_ids)

None None
[EOS] 21130
21133
{'eos_token': '[EOS]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<eop>', '<eod>']}
['<eop>', '<eod>'] [21131, 21132]


In [5]:
# 添加新tokens后的编码效果
print(tokenizer.convert_tokens_to_ids(['月光', '希望', '[EOS]', '<eop>']))
print(tokenizer.tokenize(text))
print(len(tokenizer.get_vocab()))

[21128, 21129, 21130, 21131]
[' 月  光 ', '的', '[UNK]', '[PAD]', '[CLS]', '新', ' 希  望 ', '[EOS]', '<eop>']
21133


In [6]:
# 保存分词器(包括新添加的tokens)
tokenizer.save_pretrained("../extra_dataset/save_tokenizer/")

('../extra_dataset/save_tokenizer/tokenizer_config.json',
 '../extra_dataset/save_tokenizer/special_tokens_map.json',
 '../extra_dataset/save_tokenizer/vocab.txt',
 '../extra_dataset/save_tokenizer/added_tokens.json',
 '../extra_dataset/save_tokenizer/tokenizer.json')

In [7]:
# 从本地重新加载
tokenizer1 = BertTokenizer.from_pretrained("../extra_dataset/save_tokenizer/")
tokenizer1

BertTokenizer(name_or_path='../extra_dataset/save_tokenizer/', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '[EOS]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<eop>', '<eod>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	21128: AddedToken("月光", rstrip=False, lstrip=F

In [8]:
print(tokenizer1.convert_tokens_to_ids(['月光', '希望', '[EOS]', '<eop>']))
print(tokenizer1.tokenize(text))
print(len(tokenizer1.get_vocab()))

[21128, 21129, 21130, 21131]
['月光', '的', '[UNK]', '[PAD]', '[CLS]', '新', '希望', '[EOS]', '<eop>']
21133


In [9]:
model = AutoModel.from_pretrained('bert-base-chinese')

In [10]:
model(**tokenizer(text, return_tensors='pt'))  # 报错

IndexError: index out of range in self

In [ ]:
model.embeddings

In [ ]:
# word_embedding维度为:21128 * 768(but此时tokenizer大小为:21133)
model.get_input_embeddings()

In [ ]:
# Resizes input token embeddings matrix of the model if `new_num_tokens != config.vocab_size`.
model.resize_token_embeddings(len(tokenizer))

In [ ]:
model(**tokenizer(text, return_tensors='pt'))